In [1]:
# 3 variant of kinoparse - pretty well working variant

# Get infrormation - parse

In [1]:
#работа с классами ul-весь список на странице li отдельная табличка рассмотрим все страницы
#тут мы парсим первую страницу сортированную по 10, а потом парсим по 25 след все умещатеся в 1 цикл
import requests
from bs4 import BeautifulSoup as bs
import time
import csv

In [ ]:
headers={'accept': '*/*',
          'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.143 YaBrowser/19.7.1.114 Yowser/2.5 Safari/537.36'}   
#это берем с начальной страницы браузера который юзаем через панель разраба

user_id  = 23686731 #int(input())
base_url='https://www.kinopoisk.ru/user/%d/movies/list/sort/default/vector/desc/perpage/25/#list'%(user_id)

#https://www.kinopoisk.ru/user/23686731/movies/list/sort/default/vector/desc/page/1/#list

#WTF ??? https://www.kinopoisk.ru/mykp/movies/list/type/3575/sort/default/vector/desc/vt/all/page/2/
#old - https://www.kinopoisk.ru/user/%d/movies/list/sort/default/vector/desc/perpage/10/#list

In [3]:
def kino_parse(base_url,headers):
    Kino=[]
    
    session=requests.Session()#for contionious searching,web-side think that we are human
    request=session.get(base_url,headers=headers)
    
    if request.status_code == 200:#chech does server give us data
        print('oke\n')
        
        soup=bs(request.content, 'html.parser')
        #print(soup)
                
        try:
            stranichi= soup.find('div',attrs={'class':'navigator'})#получили строку со страницами
            #stranichi= soup.find('ul',attrs={'class':'list'})
            #print('in try',stranichi,'\n')

            urleeees=stranichi.find_all('li',attrs={'class':'arr'})#нашли в строке со страницами кнопки перехода на след и послед страницу
            #print('in try',len(urleeees),'\n')
            #print('in try',urleeees[0],urleeees[-1],'\n')

            urleeees2=urleeees[-1].find('a').get('href')#получили adress последней страницы
            print('last page: ',urleeees2,' ',type(urleeees2),'\n')

            urleeees2='https://www.kinopoisk.ru' + urleeees2 #получили полную ссылку на последнюю страницу
            #/user/23686731/movies/list/sort/default/vector/desc/page/5/#list
            print('urleeees2_final:',urleeees2)

            step=1 #переход по страницам
            #!!!!!!!!!!!!!!!!!
            #base_url='https://www.kinopoisk.ru/user/23686731/movies/list/sort/default/vector/desc/page/1/#list' #%(user_id)
            #!!!!!!!!!!!!!!!!! 
            
            #---------------------------------------------------------------------------------------------------------
            while base_url!=urleeees2:

                session=requests.Session()#for contionious searching,web-side think that we are human
                request=session.get(base_url,headers=headers)
                #-------------------------------------------------------------------------------------
                soup=bs(request.content, 'html.parser')

                ma_list=soup.find('ul',attrs={'id':'itemList'})
                #print(type(ma_list))
                print('длинна листа/таблицы:',len(ma_list))
                #print(ma_list,'\n')

                li_s=ma_list.find_all('li') #ma_list.find('li')
                print(len(li_s),'\n')
                #-------------------------------------------------------------------------------------
                for li in li_s:
                    title = li.find('a', attrs={'class': 'name' }).text
                    title_orig = li.find('span').text
                    #href=li.find('a', attrs={'class': 'name' })['href']#links for titles
                    rezhiser=li.find('a', attrs={'class': 'lined' }).text
                    try:
                        rating=li.find('div', attrs={'class': 'rating' }).text
                        print(type(rating))
                        if type(rating)==str:
                            iii=0
                            c=''
                            while rating[iii]!=' ':
                                c+=rating[iii]
                                iii+=1
                            rating=float(c)
                        else:
                            #print(type(rating))
                            pass
                        
                    except:
                        rating=0

                    print(title,' ',title_orig,' ',rezhiser,' ',rating,' ','\n')

                    Kino.append({'title':title,'title_orig':title_orig,'rezhiser':rezhiser,'rating':rating})
                #-------------------------------------------------------------------------------------
                
                step+=1
                if base_url=='https://www.kinopoisk.ru/user/23686731/movies/list/sort/default/vector/desc/perpage/10/#list':
                    print('first page\n')
                    base_url='https://www.kinopoisk.ru/user/23686731/movies/list/sort/default/vector/desc/page/2/#list'
                    continue
                else:
                    base_url='https://www.kinopoisk.ru/user/23686731/movies/list/sort/default/vector/desc/page/%d/#list'%(step)
                    print('other case\n')

            #---------------------------------------------------------------------------------------------------------
            
            #тут, возможно, дописать цикл для последней страницы
            session=requests.Session()#for contionious searching,web-side think that we are human
            request=session.get(base_url,headers=headers)
            #-------------------------------------------------------------------------------------
            soup=bs(request.content, 'html.parser')

            ma_list=soup.find('ul',attrs={'id':'itemList'})
            #print(type(ma_list))
            print(len(ma_list))
            #print(ma_list,'\n')

            li_s=ma_list.find_all('li') #ma_list.find('li')
            print(len(li_s),'\n')
            #-------------------------------------------------------------------------------------
            for li in li_s:
                title = li.find('a', attrs={'class': 'name' }).text
                title_orig = li.find('span').text
                #href=li.find('a', attrs={'class': 'name' })['href']#links for titles
                rezhiser=li.find('a', attrs={'class': 'lined' }).text
                try:
                    rating=li.find('div', attrs={'class': 'rating' }).text
                    print(type(rating))
                    if type(rating)==str:
                        iii=0
                        c=''
                        while rating[iii]!=' ':
                            c+=rating[iii]
                            iii+=1
                        rating=float(c)
                    else:
                        #print(type(rating))
                        pass

                except:
                    rating=0

                print(title,'\t',title_orig,'\t',rezhiser,'\t',rating,'\t','\n')

                Kino.append({'title':title,'title_orig':title_orig,'rezhiser':rezhiser,'rating':rating})
                  
        except:
            print('\n','there is only 1 page','\n')
        
    else:
        print('error')
        
    return Kino


oke

last page:  /user/23686731/movies/list/sort/default/vector/desc/page/2/#list   <class 'str'> 

urleeees2_final: https://www.kinopoisk.ru/user/23686731/movies/list/sort/default/vector/desc/page/2/#list
длинна листа/таблицы: 51
25 

<class 'str'>
Школа-тюрьма (мини-сериал)   Kangoku Gakuen: Prison School (2015) 30 мин.   Нобору Игути   6.118   

<class 'str'>
Моя первая гяру (сериал)   Hajimete No Gal (2017) 23 мин.   Хироюки Фурукава   6.279   

<class 'str'>
Смертельный укус (сериал)   Killing Bites (2018) 24 мин.   Ясуто Нисиката   6.91   

<class 'str'>
Мстители: Финал   Avengers: Endgame (2019) 181 мин.   Энтони Руссо   7.651   

<class 'str'>
Джей и молчаливый Боб наносят ответный удар   Jay and Silent Bob Strike Back (2001) 104 мин.   Кевин Смит   7.153   

<class 'str'>
Тусовщики из супермаркета   Mallrats (1995) 94 мин.   Кевин Смит   7.103   

<class 'str'>
Настоящий детектив (сериал)   True Detective (2014 – ...) 60 мин.   Кэри Фукунага   8.705   

<class 'str'>
Необъятны

In [ ]:
def files_writer(kinos):
    with open('parse_kinos.csv','w',encoding='utf8') as file:# 'a'- append,'w'- write
        a_pen=csv.writer(file)
        a_pen.writerow(('Название фильма', 'Название фильма(оригинал)', 'Режиссер', 'Рейтинг'))
        for kino in kinos:
            a_pen.writerow((kino['title'], kino['title_orig'], kino['rezhiser'], kino['rating']))
            

In [ ]:
#kino_parse(base_url,headers)
kinos=kino_parse(base_url,headers)

#for i in kinos:
#    print(i)
           
files_writer(kinos)
#https://www.kinopoisk.ru/user/23686731/movies/list/sort/default/vector/desc/perpage/10/#list
#https://www.kinopoisk.ru/user/23686731/movies/list/sort/default/vector/desc/page/2/#list

In [2]:
#tut will be read file xlsx  or csv
#https://edwvb.blogspot.com/2016/08/kak-v-python-chitat-fajly-formata-excel.html
#https://python-scripts.com/import-csv-python

#ПОКА МЫ ДОЛЖНЫ ПЕРЕДЕЛЫВАТЬ csv ФАЙЛ В xlsx ФАЙЛ А ПОТОМ ИЗ НЕГО ЧИТАТЬ
import xlrd

rb = xlrd.open_workbook(r'C:\Users\poll\zzz_Parsing\Kniga1_kino_parse3.xlsx',formatting_info=False)
sheet = rb.sheet_by_index(0)

listik = []
for i in range(sheet.nrows):
    r = sheet.row_values(i)
    listik.append(r)
#print (listik)
listik.pop(0)
#------------------------------------------------------------------------перевод строки в столбце оценка в числа
ii=0
for i in listik:
    #print(i,'\n')
    #print(type(i[3]))
    if type(i[3])==str:
        #print(ii,i)
        iii=0
        c=''
        for iii in range(len(listik[ii][3])):
            c+=listik[ii][3][iii]
            #print(iii)
        listik[ii][3]=float(c)
        #print(listik[ii][3],'\n')
    else:
        #print(type(i[3]))
        pass
    #print(i,'\n')
    ii+=1   

for i in listik:
    print(i,'\n')
print(len(listik))
    
#========================================================================
#========================================================================

'''но как быть с первой строчкой? она 
import csv
 
def csv_dict_reader(file_obj):
    """
    Read a CSV file using csv.DictReader
    """
    reader = csv.DictReader(file_obj, delimiter=',')
    for line in reader:
        print(line["title"]),
        print(line["title_orig"]),
        print(line["rezhiser"]),
        print(line["rating"])
 
if __name__ == "__main__":
    with open("data.csv") as f_obj:
        csv_dict_reader(f_obj)


{'title':title,'title_orig':title_orig,'rezhiser':rezhiser,'rating':rating}
'''

['Школа-тюрьма (мини-сериал)', 'Kangoku Gakuen: Prison School (2015) 30 мин.', 'Нобору Игути', 6.118] 

['Моя первая гяру (сериал)', 'Hajimete No Gal (2017) 23 мин.', 'Хироюки Фурукава', 6.279] 

['Смертельный укус (сериал)', 'Killing Bites (2018) 24 мин.', 'Ясуто Нисиката', 6.91] 

['Мстители: Финал', 'Avengers: Endgame (2019) 181 мин.', 'Энтони Руссо', 7.651] 

['Джей и молчаливый Боб наносят ответный удар', 'Jay and Silent Bob Strike Back (2001) 104 мин.', 'Кевин Смит', 7.153] 

['Тусовщики из супермаркета', 'Mallrats (1995) 94 мин.', 'Кевин Смит', 7.103] 

['Настоящий детектив (сериал)', 'True Detective (2014 – ...) 60 мин.', 'Кэри Фукунага', 8.705] 

['Необъятный океан (сериал)', 'Grand Blue (2018) 25 мин.', 'Синдзи Такамацу', 7.799] 

['Рыбка-бананка (сериал)', 'Banana Fish (2018) 25 мин.', 'Уцуми Хироко', 7.606] 

['Ад Данте: Анимированный эпос', "Dante's Inferno: An Animated Epic (2010) 84 мин.", 'Виктор Кук', 7.016] 

['D: Жажда крови', 'Vampire Hunter D: Bloodlust (2000) 103 

'но как быть с первой строчкой? она \nimport csv\n \ndef csv_dict_reader(file_obj):\n    """\n    Read a CSV file using csv.DictReader\n    """\n    reader = csv.DictReader(file_obj, delimiter=\',\')\n    for line in reader:\n        print(line["title"]),\n        print(line["title_orig"]),\n        print(line["rezhiser"]),\n        print(line["rating"])\n \nif __name__ == "__main__":\n    with open("data.csv") as f_obj:\n        csv_dict_reader(f_obj)\n\n\n{\'title\':title,\'title_orig\':title_orig,\'rezhiser\':rezhiser,\'rating\':rating}\n'

# Customizing our results by plots

In [3]:
#here MNK method
import numpy as np

def MNK (x, y, F_x, NxN=1):
    N=NxN+1
    X_matrix=np.zeros((N,N))
    Y_matrix=np.zeros(N)
    A_i_matrix=np.zeros(N)
    
    F_ans=np.zeros(len(F_x))
    
    #print('degree of polinom: ',NxN)
    for i in range(N):
        kk_i=NxN-i
        for j in range(N):
            #kk=N-i-j #степень в миноре икса
            kk_j=NxN-j
            for n in range(len(x)):
                X_matrix[i][j]+=(x[n]**kk_i)*(x[n]**kk_j)
    
    for i in range(N):
        kk_i=NxN-i
        for n in range(len(y)):
                Y_matrix[i]+=(y[n])*(x[n]**kk_i)
                
    print(X_matrix,'*(a)=',Y_matrix)                
    A_i_matrix = np.linalg.solve(X_matrix, Y_matrix)
    
    for n in range(len(F_ans)):
        for i in range(len(A_i_matrix)):
            kk_i=NxN-i
            F_ans[n]+=A_i_matrix[i]*(F_x[n]**kk_i)
            
    return F_ans
    

In [4]:
from matplotlib import pylab as plt
from matplotlib import rcParams
%matplotlib inline
#rcParams['figure.dpi'] = 140

#listik

x_m=[]
y_m=[]
num_of_dots=200 #int(input()) # сколько всего будет точек на графике 200
print('Enter the degree of the curvature')
degree=int(input())
print('\n')

for i in range(len(listik)):
    x_m.append(i)
    y_m.append(listik[i][-1])
    #print(x_m[i],' ',listik[i][0],' ',y_m[i])
        
print(x_m,'\n',y_m)

f_x_m=np.linspace(x_m[0],x_m[-1],num_of_dots)
f_y_m=MNK(x_m,y_m,f_x_m,degree)
    
'''
plt.title('Kino')
plt.ylabel('rate(x/10)')
plt.xlabel('Film number')
plt.grid(True)   
plt.plot(f_x_m,f_y_m,color='black')
#xdel.clear()
#ydel.clear()
plt.show()
'''

Enter the degree of the curvature
1


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43] 
 [6.118, 6.279, 6.91, 7.651, 7.153, 7.103, 8.705, 7.799, 7.606, 7.016, 7.726, 0.0, 0.0, 0.0, 8.006, 0.0, 7.126, 7.273, 7.351, 8.106, 7.586, 8.407, 8.372, 0.0, 7.396, 7.512, 8.285, 7.667, 8.442, 7.425, 8.112, 7.508, 7.866, 8.07, 8.124, 7.637, 7.587, 7.358, 6.773, 7.626, 7.627, 7.54, 8.56, 7.093]
[[27434.   946.]
 [  946.    44.]] *(a)= [6719.748  296.501]


"\nplt.title('Kino')\nplt.ylabel('rate(x/10)')\nplt.xlabel('Film number')\nplt.grid(True)   \nplt.plot(f_x_m,f_y_m,color='black')\n#xdel.clear()\n#ydel.clear()\nplt.show()\n"

In [5]:
import bokeh

In [6]:
from bokeh.plotting import figure, ColumnDataSource #,LabelSet 
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models.tools import HoverTool
#from bokeh.layouts import row

import pandas as pd
df=pd.read_csv('parse_kinos.csv')
source=ColumnDataSource(df)#source = ColumnDataSource(dict(x=x, y=y, sizes=sizes))

src = ColumnDataSource(df)
src.data.keys()
print(src.data.keys())

xx=df['Название фильма']
yy=df['Рейтинг']
#print(yy)

output_notebook()#"log_lines2.html"

rate_list = source.data['Рейтинг'].tolist()
#print(rate_list)
p = figure(
    y_range=[-1,11],
    plot_width=1000,
    plot_height=350,
    title='Рейтинг фильмов',
    y_axis_label='Оценка х/10',
    x_axis_label='фильмы'
)

#firebrick
#darkred

#p3 = figure(**opts)
#p = figure()
r3 = p.circle('index', 'Рейтинг', source=source, legend="Film", fill_color="firebrick",line_color="black", size=8)
#r3 = p.circle(x_m, y_m, legend="Film", fill_color="white", size=8)

#p4 = figure(**opts)
r4 = p.line(f_x_m, f_y_m,line_color="darkcyan")


hover=HoverTool()
hover.tooltips= [
    ("Название фильма", "@{Название фильма}"),
    ("Название фильма(оригинал)", "@{Название фильма(оригинал)}"),
    ("Режиссер", "@{Режиссер}"),
    ("Рейтинг", "@{Рейтинг}")
]

#{'title':title,'title_orig':title_orig,'rezhiser':rezhiser,'rating':rating}
#('Название фильма', 'Название фильма(оригинал)', 'Режиссер', 'Рейтинг')

p.add_tools(hover)
# get a handle to update the shown cell with
#проблема в том что у меня при не нулевых н там еще стр файлы лежат нужно чтобы там были только флоат данные
t2 = show(p, notebook_handle=False)

dict_keys(['index', 'Название фильма', 'Название фильма(оригинал)', 'Режиссер', 'Рейтинг'])


Loading BokehJS ...

In [7]:
#можно заменить икс на год выпуска фмльма

In [8]:
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook

output_notebook()#"log_lines2.html"
opts = dict(plot_width=1000, plot_height=350, min_border=0)

#p3 = figure(**opts)
p = figure(**opts)
r3 = p.circle(x_m, y_m, legend="Film", fill_color="white", size=8)

#p4 = figure(**opts)
r4 = p.line(f_x_m, f_y_m,line_color="red")

# get a handle to update the shown cell with
t2 = show(p, notebook_handle=False)

Loading BokehJS ...

In [9]:
from bokeh.plotting import figure, output_file, show

# prepare some data
#x = [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
#y0 = [i**2 for i in x]
#y1 = [10**i for i in x]
#y2 = [10**(i**2) for i in x]

# output to static HTML file
output_file("log_lines2.html")
#output_notebook() 

# create a new plot
p = figure(
   tools="pan,box_zoom,reset,save",
   title="Kinopoisk Rating",
   x_axis_label='Film number', y_axis_label='Rating'
)

# add some renderers
#p.line(x_m, y_m, legend="y=x")
p.circle(x_m, y_m, legend="y=x", fill_color="white", size=8)

# show the results
show(p)


In [10]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show

output_notebook()

plot = figure()
plot.circle(x_m, y_m, legend="y=x", fill_color="white", size=8)

handle = show(plot, notebook_handle=True)

# Update the plot title in the earlier cell
plot.title.text = "New Title"
push_notebook(handle=handle)

Loading BokehJS ...